# Composite nodes

One of the convenient properties of Forney-style factor graphs (as opposed to regular factor graphs) is that they naturally allow for composability: one can draw a box around part of a FFG and treat this box as a new type of factor node. For example, one can combine a gain (multiplication) node with the addition node in a so-called *composite node*, as depicted in the following graph (Fig. 4.2 from [1]):

<img src="./figures/information_filter.png" width="350"/>

Composite nodes are useful for two reasons:

1. Building large graphs becomes more convenient by 'packaging' repetitive parts of the graph as composite nodes.
2. One can define 'shortcut rules' for message updates, which might be more efficient and/or numerically stable than performing vanilla message passing on the internals of the composite node. The shortcut rule might for example exploit the matrix inversion lemma, or involve some optimization algorithm. For example, in the schedule shown above, message (4) is calculated directly from messages (2) and (3), and the message out of the gain node does not have to be represented explicitly.


Composite nodes in ForneyLab
--------------------------------

ForneyLab makes it easy to define composite nodes as well as shortcut rules. If no suitable shortcut rule is available, ForneyLab automatically reverts to explicit message passing on the internals of the composite node.

To demonstrate the use of composite nodes, we'll build a factor graph involving a gain-addition combination like in the "information filter" factor graph shown above.

### 1. 'Flat' factor graph without composite node

In [1]:
using ForneyLab

# Define factor graph for x1 = x0 + b*u1, where x0 and u1 have Gaussian priors, and b is a constant.
# This is a part of the information filter graph from the introduction.
fg = FactorGraph()
@RV x0 ~ GaussianMeanVariance(constant(ones(2)), constant(eye(2)), id=:x0)
@RV u1 ~ GaussianMeanVariance(constant(1.0), constant(1.0), id=:u)
b = constant([1.0; 0.5], id=:b)
@RV x1 = x0 + b*u1; x1.id=:x1;

Let's create a sum-product message passing schedule to calculate $p(x_1)$:

In [2]:
flat_schedule = sumProductSchedule(x1)
draw(schedule=flat_schedule)
show(flat_schedule)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.36.0 (20140111.2315)
 -->
<!-- Title: G Pages: 1 -->
 
 
 G 
 
<!-- 9112215565128108659 -->
 9112215565128108659 
 
 𝒩 
 x0 
 
<!-- 7066188827649088811 -->
 7066188827649088811 
 
 clamp_1 
 
<!-- 9112215565128108659--7066188827649088811 -->
 9112215565128108659--7066188827649088811 
 
 1 out 
 
<!-- 15306651246088901811 -->
 15306651246088901811 
 
 clamp_2 
 
<!-- 9112215565128108659--15306651246088901811 -->
 9112215565128108659--15306651246088901811 
 
 1 out 
 
<!-- 9206292118644337480 -->
 9206292118644337480 
 
 clamp_3 
 
<!-- 11281585512770440673 -->
 11281585512770440673 
 
 clamp_4 
 
<!-- 4791361569344842402 -->
 4791361569344842402 
 
 𝒩 
 u 
 
<!-- 4791361569344842402--9206292118644337480 -->
 4791361569344842402--9206292118644337480 
 
 1 out 
 
<!-- 4791361569344842402--11281585512770440673 -->
 4791361569344842402--11281585512770440673 
 
 1 out 
 
<!-- 13328539673264492846 -->
 13328539673264492846 
 
 + 
 addition_1 
 
<!-- 13328539673264492846--9112215565128108659 -->
 13328539673264492846--9112215565128108659 
 
 1 out 
 (3) 
 
<!-- 10363527005759130447 -->
 10363527005759130447 
 
 × 
 multiplication_1 
 
<!-- 13328539673264492846--10363527005759130447 -->
 13328539673264492846--10363527005759130447 
 
 1 out 
 (2) 
 
<!-- 12399756723105681130 -->
 12399756723105681130 
 
 b 
 
<!-- 10363527005759130447--4791361569344842402 -->
 10363527005759130447--4791361569344842402 
 
 1 out 
 (1) 
 
<!-- 10363527005759130447--12399756723105681130 -->
 10363527005759130447--12399756723105681130 
 
 1 out 
 
<!-- 129697889210947388412 -->
 129697889210947388412 
 
<!-- 129697889210947388412--13328539673264492846 -->
 129697889210947388412--13328539673264492846 
 
 1 out 
 (4)

	SPClamp{Multivariate} on Interface 1 (out) of ForneyLab.Clamp{ForneyLab.Multivariate} b
	SPClamp{Univariate} on Interface 1 (out) of ForneyLab.Clamp{ForneyLab.Univariate} clamp_3
	SPClamp{Univariate} on Interface 1 (out) of ForneyLab.Clamp{ForneyLab.Univariate} clamp_4
1.	SPGaussianMeanVarianceOutVPP on Interface 1 (out) of ForneyLab.GaussianMeanVariance u
2.	SPMultiplicationOutVPG on Interface 1 (out) of ForneyLab.Multiplication multiplication_1
	SPClamp{Multivariate} on Interface 1 (out) of ForneyLab.Clamp{ForneyLab.Multivariate} clamp_1
	SPClamp{MatrixVariate} on Interface 1 (out) of ForneyLab.Clamp{ForneyLab.MatrixVariate} clamp_2
3.	SPGaussianMeanVarianceOutVPP on Interface 1 (out) of ForneyLab.GaussianMeanVariance x0
4.	SPAdditionOutVGG on Interface 1 (out) of ForneyLab.Addition addition_1


As expected, we get a schedule with 4 messages. Let's now 'draw a box' around the multiplication and addition nodes, and create a composite node that implements the relationship $f(x_0, x_1, u_1) = \delta(x_1 - (x_0 + b*u_1))$.

### 2. Define composite node and use it in a factor graph

We can easily define a composite node using ForneyLab's `@composite` macro. It works like this:

In [3]:
# Define a composite node for z = x + b*y
@composite GainAddition (z, x, y) begin
    # Specify the 'internal factor graph' of the GainAddion composite node.
    # z, x, and y can be used as if they are existing Variables in this block.
    b = constant([1.0; 0.5], id=:b)
    Addition(z, x, b*y) # z = x + b*y
end

In the above code, `GainAddition` is the name of the composite node that we're defining. The `(z, x, y)` part defines the names of the interfaces of this node. Every interface corresponds to a `Variable` in the internal graph of the composite node, so we can use `z`, `x`, and `y` as if they are existing variables when building the internal graph.

Now that our custom `GainAddition` composite node is defined, we can use it like any other factor node. Let's build the same factor graph as before, but now with our composite node:

In [4]:
fg2 = FactorGraph()
@RV x0 ~ GaussianMeanVariance(constant(ones(2)), constant(eye(2)), id=:x0)
@RV u1 ~ GaussianMeanVariance(constant(1.0), constant(1.0), id=:u)
x1 = Variable(id=:x1)
ga_node = GainAddition(x1, x0, u1)

composite_schedule = sumProductSchedule(x1)
draw(schedule=composite_schedule)
show(composite_schedule)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.36.0 (20140111.2315)
 -->
<!-- Title: G Pages: 1 -->
 
 
 G 
 
<!-- 12778612803558611424 -->
 12778612803558611424 
 
 clamp_1 
 
<!-- 13926237203127137379 -->
 13926237203127137379 
 
 clamp_2 
 
<!-- 11720943017230657950 -->
 11720943017230657950 
 
 𝒩 
 u 
 
<!-- 7602502102886808805 -->
 7602502102886808805 
 
 clamp_3 
 
<!-- 11720943017230657950--7602502102886808805 -->
 11720943017230657950--7602502102886808805 
 
 1 out 
 
<!-- 15531141773719676475 -->
 15531141773719676475 
 
 clamp_4 
 
<!-- 11720943017230657950--15531141773719676475 -->
 11720943017230657950--15531141773719676475 
 
 1 out 
 
<!-- 4061737998791509715 -->
 4061737998791509715 
 
 GainAddition 
 gainaddition_1 
 
<!-- 4061737998791509715--11720943017230657950 -->
 4061737998791509715--11720943017230657950 
 
 1 out 
 (1) 
 
<!-- 8051032323826008169 -->
 8051032323826008169 
 
 𝒩 
 x0 
 
<!-- 4061737998791509715--8051032323826008169 -->
 4061737998791509715--8051032323826008169 
 
 1 out 
 (2) 
 
<!-- 8051032323826008169--12778612803558611424 -->
 8051032323826008169--12778612803558611424 
 
 1 out 
 
<!-- 8051032323826008169--13926237203127137379 -->
 8051032323826008169--13926237203127137379 
 
 1 out 
 
<!-- 51438706066269250982 -->
 51438706066269250982 
 
<!-- 51438706066269250982--4061737998791509715 -->
 51438706066269250982--4061737998791509715 
 
 1 z 
 (3)

	SPClamp{Univariate} on Interface 1 (out) of ForneyLab.Clamp{ForneyLab.Univariate} clamp_3
	SPClamp{Univariate} on Interface 1 (out) of ForneyLab.Clamp{ForneyLab.Univariate} clamp_4
1.	SPGaussianMeanVarianceOutVPP on Interface 1 (out) of ForneyLab.GaussianMeanVariance u
	SPClamp{Multivariate} on Interface 1 (out) of ForneyLab.Clamp{ForneyLab.Multivariate} clamp_1
	SPClamp{MatrixVariate} on Interface 1 (out) of ForneyLab.Clamp{ForneyLab.MatrixVariate} clamp_2
2.	SPGaussianMeanVarianceOutVPP on Interface 1 (out) of ForneyLab.GaussianMeanVariance x0
3.	(INTERNAL SCHEDULE) SPAdditionOutVGG on Interface 1 (z) of GainAddition gainaddition_1


As you can see, the factor graph just got simpler, and the message passing schedule only has 3 entries. However, message (3) is special, since it is produced by executing a message passing schedule on the internal graph of the composite node `gainaddition_1`. We can inspect the internal graph of the composite node and the internal message passing schedule that produces message (3) of the main schedule:

In [5]:
draw(ga_node.inner_graph, schedule=composite_schedule[end].internal_schedule)
show(composite_schedule[end].internal_schedule)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.36.0 (20140111.2315)
 -->
<!-- Title: G Pages: 1 -->
 
 
 G 
 
<!-- 13494684146027374735 -->
 13494684146027374735 
 
 Terminal y 
 
<!-- 2210198081400783474 -->
 2210198081400783474 
 
 × 
 multiplication_1 
 
<!-- 13494684146027374735--2210198081400783474 -->
 13494684146027374735--2210198081400783474 
 
 3 in2 
 
<!-- 979371016989655167 -->
 979371016989655167 
 
 + 
 addition_1 
 
<!-- 979371016989655167--2210198081400783474 -->
 979371016989655167--2210198081400783474 
 
 1 out 
 (1) 
 
<!-- 7217073831323716320 -->
 7217073831323716320 
 
 Terminal x 
 
<!-- 7217073831323716320--979371016989655167 -->
 7217073831323716320--979371016989655167 
 
 2 in1 
 
<!-- 14457931805738030308 -->
 14457931805738030308 
 
 b 
 
<!-- 2210198081400783474--14457931805738030308 -->
 2210198081400783474--14457931805738030308 
 
 1 out 
 
<!-- 3889153819588844268 -->
 3889153819588844268 
 
 Terminal z 
 
<!-- 3889153819588844268--979371016989655167 -->
 3889153819588844268--979371016989655167 
 
 1 out 
 (2)

	SPClamp{Multivariate} on Interface 1 (out) of ForneyLab.Clamp{ForneyLab.Multivariate} b
1.	SPMultiplicationOutVPG on Interface 1 (out) of ForneyLab.Multiplication multiplication_1
2.	SPAdditionOutVGG on Interface 1 (out) of ForneyLab.Addition addition_1


As you can see, each interface of the `GainAddition` node corresponds to a `Terminal` node in its internal graph. The schedule that is executed on the internal graph consists of 2 messages. 

So now we have a message passing schedule that is hierarchical: the last message in `composite_schedule` is produced by another message passing algorithm (executed on the internals of the composite node). However, when we actually generate code for this message passing schedule, the hierarchical schedule will be flattened to get a simple, linear schedule:

In [6]:
# Print message passing algorithm for the factor graph with the GainAddition composite node
println(messagePassingAlgorithm(composite_schedule))

function step!(data::Dict, marginals::Dict=Dict(), messages::Vector{Message}=Array{Message}(3))

messages[1] = ruleSPGaussianMeanVarianceOutVPP(nothing, Message(Univariate, PointMass, m=1.0), Message(Univariate, PointMass, m=1.0))
messages[2] = ruleSPGaussianMeanVarianceOutVPP(nothing, Message(Multivariate, PointMass, m=[1.0, 1.0]), Message(MatrixVariate, PointMass, m=[1.0 0.0; 0.0 1.0]))
messages[3] = ruleSPMultiplicationOutVPG(nothing, Message(Multivariate, PointMass, m=[1.0, 0.5]), messages[1])
messages[4] = ruleSPAdditionOutVGG(nothing, messages[2], messages[3])


return marginals

end


In fact, the flattened algorithm is **IDENTICAL** (up to unimportant reordering of the messages) to the algorithm that we get if we don't use the composite node! Check for yourself...

In [7]:
# Print message passing algorithm for the 'flat' factor graph WITHOUT the GainAddition composite node
println(messagePassingAlgorithm(flat_schedule))

function step!(data::Dict, marginals::Dict=Dict(), messages::Vector{Message}=Array{Message}(4))

messages[1] = ruleSPGaussianMeanVarianceOutVPP(nothing, Message(Univariate, PointMass, m=1.0), Message(Univariate, PointMass, m=1.0))
messages[2] = ruleSPMultiplicationOutVPG(nothing, Message(Multivariate, PointMass, m=[1.0, 0.5]), messages[1])
messages[3] = ruleSPGaussianMeanVarianceOutVPP(nothing, Message(Multivariate, PointMass, m=[1.0, 1.0]), Message(MatrixVariate, PointMass, m=[1.0 0.0; 0.0 1.0]))
messages[4] = ruleSPAdditionOutVGG(nothing, messages[3], messages[2])


return marginals

end


This is nice: we can define and use composite nodes without having to implement any additional message update rules.
You can manually flatten schedules containing internal message passing schedules using the `flatten` function:

In [8]:
show(ForneyLab.flatten(composite_schedule))

	SPClamp{Univariate} on Interface 1 (out) of ForneyLab.Clamp{ForneyLab.Univariate} clamp_3
	SPClamp{Univariate} on Interface 1 (out) of ForneyLab.Clamp{ForneyLab.Univariate} clamp_4
1.	SPGaussianMeanVarianceOutVPP on Interface 1 (out) of ForneyLab.GaussianMeanVariance u
	SPClamp{Multivariate} on Interface 1 (out) of ForneyLab.Clamp{ForneyLab.Multivariate} clamp_1
	SPClamp{MatrixVariate} on Interface 1 (out) of ForneyLab.Clamp{ForneyLab.MatrixVariate} clamp_2
2.	SPGaussianMeanVarianceOutVPP on Interface 1 (out) of ForneyLab.GaussianMeanVariance x0
	SPClamp{Multivariate} on Interface 1 (out) of ForneyLab.Clamp{ForneyLab.Multivariate} b
3.	SPMultiplicationOutVPG on Interface 1 (out) of ForneyLab.Multiplication multiplication_1
4.	SPAdditionOutVGG on Interface 1 (out) of ForneyLab.Addition addition_1


### 3. Defining shortcut rules

If we actually want to use composite nodes to get a different (i.e. more efficient) algorithm, we'll have to specify message update rules that apply specifically to the composite nodes at hand. These rules are sometimes referred to as *shortcut rules*, since they provide a way to shortcut the calculation of internal messages.

As an example, we'll specify a custom shortcut rule to calculate the (Gaussian) sum-product message towards the `z` interface. There is nothing special about the rule definitions for composite nodes, they are identical to the rule definitions for regular nodes.

In [9]:
@sumProductRule(:node_type     => GainAddition,                                 # our custom composite node
                :outbound_type => Message{Gaussian},                            # this rule produces a Gaussian msg
                :inbound_types => (Void, Message{Gaussian}, Message{Gaussian}), # msg towards first interface, incoming types
                :name          => SPGainAdditionOutVGG)                         # name of the update rule;

If we now build a new sum-product algorithm, the custom shortcut rule will automatically be picked up:

In [10]:
shortcut_schedule = sumProductSchedule(x1)
show(shortcut_schedule)

	SPClamp{Univariate} on Interface 1 (out) of ForneyLab.Clamp{ForneyLab.Univariate} clamp_3
	SPClamp{Univariate} on Interface 1 (out) of ForneyLab.Clamp{ForneyLab.Univariate} clamp_4
1.	SPGaussianMeanVarianceOutVPP on Interface 1 (out) of ForneyLab.GaussianMeanVariance u
	SPClamp{Multivariate} on Interface 1 (out) of ForneyLab.Clamp{ForneyLab.Multivariate} clamp_1
	SPClamp{MatrixVariate} on Interface 1 (out) of ForneyLab.Clamp{ForneyLab.MatrixVariate} clamp_2
2.	SPGaussianMeanVarianceOutVPP on Interface 1 (out) of ForneyLab.GaussianMeanVariance x0
3.	SPGainAdditionOutVGG on Interface 1 (z) of GainAddition gainaddition_1


Yes, the new schedule uses the shortcut rule, allowing it to calculate the result in just 3 message updates instead of 4!

References
------------

[1] S. Korl, "A Factor Graph Approach to Signal Modelling, System Identification and Filtering", 2005.